# PRICING STRATEGY

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm

In [2]:
data = pd.read_csv("./data/raw/Breakfast At The Frat_dataset/cereal_merged_df.csv")
data


,Unnamed: 0,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,...,PRODUCT_SIZE,STORE_ID,STORE_NAME,ADDRESS_CITY_NAME,ADDRESS_STATE_PROV_CODE,MSA_CODE,SEG_VALUE_NAME,PARKING_SPACE_QTY,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,0,2009-01-14,367.0,1111085319,14.0,13.0,13.0,26.32,1.88,1.88,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
1,1,2009-01-21,367.0,1111085319,12.0,12.0,12.0,22.68,1.89,1.89,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
2,2,2009-01-28,367.0,1111085319,18.0,17.0,16.0,33.66,1.87,1.87,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
3,3,2009-02-04,367.0,1111085319,13.0,13.0,13.0,24.44,1.88,1.88,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
4,4,2009-02-11,367.0,1111085319,16.0,16.0,16.0,29.92,1.87,1.87,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169684,169684,2011-12-07,29159.0,88491212971,12.0,12.0,12.0,34.20,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169685,169685,2011-12-14,29159.0,88491212971,14.0,14.0,14.0,39.90,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169686,169686,2011-12-21,29159.0,88491212971,22.0,22.0,21.0,62.70,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169687,169687,2011-12-28,29159.0,88491212971,17.0,16.0,16.0,48.45,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051


In [3]:
family = data.loc[data['SUB_CATEGORY']=='ALL FAMILY CEREAL']
kids= data.loc[data['SUB_CATEGORY']=='KIDS CEREAL']

family_price = family.PRICE.mean()
kids_price = kids.PRICE.mean()


In [4]:
# for preparing data for regression 
def data_prep(data):
    # kellog
    df = data.loc[data['MANUFACTURER']=="KELLOGG"].groupby(['WEEK_END_DATE','STORE_NUM']).agg({'PRICE':'mean','UNITS':'sum'}).reset_index()
    df.rename(columns={"PRICE":'kellog_price','UNITS':'kellog_units'},inplace=True)
    
    # other brands
    mi = data.loc[data['MANUFACTURER']=="GENERAL MI"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    private = data.loc[data['MANUFACTURER']=="PRIVATE LABEL"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    post = data.loc[data['MANUFACTURER']=="POST FOODS"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    quaker = data.loc[data['MANUFACTURER']=="QUAKER"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']

    df['mi_price'] = mi
    df['private_price'] = private
    df['post_price'] = post
    df['quaker_price'] = quaker
    
    # check # of transactions for each brand 
    print('kellog:',df.shape[0])
    print('General MI:',mi.shape[0])
    print('Private Label:',private.shape[0])
    print('Post Foods:',post.shape[0])
    print('Quarker:',quaker.shape[0])
    
    # convert prices variables to log-prices
    to_log = ['kellog_units','kellog_price','mi_price','private_price','post_price','quaker_price']
    df[to_log] = np.log(df[to_log])
    
    # specify categorical variables
    df['STORE_NUM'] = df['STORE_NUM'].astype('category')
    
    return (df)

df = data_prep(family) # post doesn't do Family cereal
df = df.loc[df['quaker_price'].notnull()]
df

kellog: 11904
General MI: 11987
Private Label: 11986
Post Foods: 0
Quarker: 10232


,WEEK_END_DATE,STORE_NUM,kellog_price,kellog_units,mi_price,private_price,post_price,quaker_price
0,2009-01-14,367.0,1.144223,2.639057,1.252763,0.657520,NaN,0.712950
1,2009-01-14,387.0,1.128171,4.007333,1.184790,0.633928,NaN,0.693147
2,2009-01-14,389.0,1.249902,3.526361,1.119504,0.649718,NaN,0.698135
3,2009-01-14,613.0,1.217876,2.484907,1.256565,0.628609,NaN,0.693147
4,2009-01-14,623.0,1.057790,3.044522,1.241269,0.649718,NaN,0.693147
...,...,...,...,...,...,...,...,...
10227,2011-08-10,4521.0,1.211941,1.945910,1.278152,0.672944,NaN,0.916291
10228,2011-08-10,6179.0,1.358409,3.663562,1.274431,0.690644,NaN,0.996949
10229,2011-08-10,6187.0,1.358409,3.806662,1.291068,0.722706,NaN,1.081805
10230,2011-08-10,6379.0,1.211941,3.401197,1.294727,0.722706,NaN,1.081805


## MODEL 1: Only Price Variable

In [5]:
COGS = 0.6

def run_ols_regression(Y, X):
    X = sm.add_constant(X)  # adding a constant
    model = sm.OLS(Y, X).fit()
    return model

def cal_opt_price(price, elasticity):
    return price * elasticity/(1+elasticity)

model_1 = run_ols_regression(df['kellog_units'], df[['kellog_price', 'mi_price', 'private_price', 'quaker_price']])
print(model_1.summary())
print('Result pricing:', cal_opt_price(price=family_price*COGS,elasticity=model_1.params['kellog_price']))   

                            OLS Regression Results                            
Dep. Variable:           kellog_units   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     219.0
Date:                Wed, 26 Feb 2025   Prob (F-statistic):          1.20e-180
Time:                        19:04:48   Log-Likelihood:                -11811.
No. Observations:               10232   AIC:                         2.363e+04
Df Residuals:                   10227   BIC:                         2.367e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.1087      0.144     28.589

## MODEL 2: Price Variable + Store Num

In [6]:
df = pd.get_dummies(data=df,columns=['STORE_NUM'], drop_first = True)


In [7]:
Y = df['kellog_units']
X = df.drop(['WEEK_END_DATE','kellog_units','post_price'],axis=1).astype(float)
model_2 = run_ols_regression(Y, X)
print(model_2.summary())
print('Result pricing:', cal_opt_price(price=family_price*COGS,elasticity=model_2.params['kellog_price']))   

                            OLS Regression Results                            
Dep. Variable:           kellog_units   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     302.8
Date:                Wed, 26 Feb 2025   Prob (F-statistic):               0.00
Time:                        19:04:48   Log-Likelihood:                -5991.3
No. Observations:               10232   AIC:                         1.214e+04
Df Residuals:                   10151   BIC:                         1.273e+04
Df Model:                          80                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 4.8034      0.09

## MODEL 3: price variables + store_num + week_end_date


In [8]:
df = pd.get_dummies(data=df,columns=['WEEK_END_DATE'], drop_first = True)


In [9]:
Y = df['kellog_units']
X = df.drop(['kellog_units','post_price'],axis=1).astype(float)
model_3 = run_ols_regression(Y, X)
print(model_3.summary())
print('Result pricing:', cal_opt_price(price=family_price*COGS,elasticity=model_3.params['kellog_price']))   

                            OLS Regression Results                            
Dep. Variable:           kellog_units   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     196.3
Date:                Wed, 26 Feb 2025   Prob (F-statistic):               0.00
Time:                        19:04:49   Log-Likelihood:                -3802.2
No. Observations:               10232   AIC:                             8034.
Df Residuals:                   10017   BIC:                             9589.
Df Model:                         214                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

## Bias induced by not considering competition


In [10]:
Y=df['kellog_units']
#X=df[['kellog_price']]
X = df.drop(['kellog_units','post_price','mi_price','private_price','quaker_price'],axis=1).astype(float)
model_4 = run_ols_regression(Y, X)
print(model_4.summary())
print('Result pricing:', cal_opt_price(price=family_price*COGS,elasticity=model_4.params['kellog_price']))   

                            OLS Regression Results                            
Dep. Variable:           kellog_units   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     199.1
Date:                Wed, 26 Feb 2025   Prob (F-statistic):               0.00
Time:                        19:04:50   Log-Likelihood:                -3803.6
No. Observations:               10232   AIC:                             8031.
Df Residuals:                   10020   BIC:                             9565.
Df Model:                         211                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [11]:
with_comp = cal_opt_price(price=family_price*COGS,elasticity=model_3.params['kellog_price'])
no_comp = cal_opt_price(price=family_price*COGS,elasticity=model_4.params['kellog_price'])
bias = with_comp - no_comp
percent_bias = 100*(bias/with_comp)
round(percent_bias,4)

np.float64(-0.0664)